## Imports

In [1]:
import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('bmh')
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

In [2]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.models import Sequential
import cv2

import keras
from keras import Model
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.applications.vgg16 import VGG16
from keras.preprocessing import image
from keras.applications.vgg16 import preprocess_input
from keras.applications.vgg16 import decode_predictions
from tensorflow.keras.optimizers import SGD

---

## Data Loading and Pre-processing

In [3]:
img_npy = np.load('data_train_correct.npy')
label = np.load('labels_train_corrected.npy')

In [4]:
np.unique(label)

array([ 0.,  1.,  2.,  3.,  4.,  5.,  6.,  7.,  8.,  9., 10.])

In [5]:
train_img = []

VGG16 only takes images in 3 channels, so I need to convert npy data to 300X300 and then duplicate the channels

temp_image = img_npy[:,0].reshape((300,300))
temp_image = np.expand_dims(temp_image,-1)
temp_image_3_channel = temp_image.repeat(3,axis=-1)

In [6]:
new_dimensions = (200,200)

In [7]:
for i in range(img_npy.shape[1]):
    temp_image = img_npy[:,i].reshape((300,300))
    temp_image = np.expand_dims(temp_image,-1)
    temp_image_3_channel = temp_image.repeat(3,axis=-1)
    resized_img = cv2.resize(src=temp_image_3_channel, dsize=new_dimensions, interpolation=cv2.INTER_LINEAR)
    train_img.append(resized_img)

In [8]:
train_img = np.array(train_img)

In [9]:
train_img = preprocess_input(train_img)

In [10]:
X_train, X_valid, label_train, label_valid = train_test_split(train_img, label, test_size=0.3)

In [11]:
X_train = X_train/255
X_valid = X_valid/255

model.predict() breaks as it tries to share some work between CPU and GPU. So it needs to be executed on either 1

In [12]:
from tensorflow.python.client import device_lib

def get_available_devices():
    local_device_protos = device_lib.list_local_devices()
    return [x.name for x in local_device_protos]

GPU_names = get_available_devices()
print(GPU_names)
#with tf.device('/gpu:0'):
    # Do GPU stuff here
#with tf.device('/cpu:0'):
    # Do CPU stuff here

['/device:CPU:0', '/device:GPU:0', '/device:GPU:1']


# Extracting features from the train dataset using the VGG16 pre-trained model
with tf.device('/cpu:0'):
    features_train=model.predict(train_img, batch_size=32)

In [13]:
model = VGG16(weights='imagenet', include_top=False, input_shape = (new_dimensions[0],new_dimensions[1],3), pooling='max')

In [14]:
flat1 = Flatten()(model.layers[-1].output)
class1 = Dense(1024, activation='relu')(flat1)
output = Dense(11,activation='softmax')(class1)

In [15]:
new_model = Model(inputs=model.inputs, outputs=output)

In [16]:
new_model.summary()

Model: "model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 200, 200, 3)]     0         
_________________________________________________________________
block1_conv1 (Conv2D)        (None, 200, 200, 64)      1792      
_________________________________________________________________
block1_conv2 (Conv2D)        (None, 200, 200, 64)      36928     
_________________________________________________________________
block1_pool (MaxPooling2D)   (None, 100, 100, 64)      0         
_________________________________________________________________
block2_conv1 (Conv2D)        (None, 100, 100, 128)     73856     
_________________________________________________________________
block2_conv2 (Conv2D)        (None, 100, 100, 128)     147584    
_________________________________________________________________
block2_pool (MaxPooling2D)   (None, 50, 50, 128)       0     

In [17]:
#setting first 8 layers to non-trianable

for layer in new_model.layers[:8]:

    layer.trainable = False

In [18]:
# Learning rate is changed to 0.001
sgd = SGD(learning_rate=0.002, decay=1e-6, momentum=0.9, nesterov=True)
new_model.compile(optimizer=sgd, loss='sparse_categorical_crossentropy', metrics=['accuracy'])
#new_model.compile(optimizer='rmsprop',loss='sparse_categorical_crossentropy', metrics=['accuracy'])


with tf.device('/cpu:0'):
    history = new_model.fit(X_train, label_train, epochs=5, batch_size=64, validation_data=(X_valid, label_valid))

In [19]:
with tf.device('/gpu:0'):
    history = new_model.fit(X_train, label_train, epochs=20, batch_size=16, validation_data=(X_valid, label_valid))

Epoch 1/20
396/396 [==============================] - 27s 59ms/step - loss: 0.5093 - accuracy: 0.8532 - val_loss: 0.2827 - val_accuracy: 0.9181
Epoch 2/20
396/396 [==============================] - 22s 56ms/step - loss: 0.2337 - accuracy: 0.9345 - val_loss: 0.3902 - val_accuracy: 0.8720
Epoch 3/20
396/396 [==============================] - 22s 56ms/step - loss: 0.1739 - accuracy: 0.9499 - val_loss: 0.2392 - val_accuracy: 0.9317
Epoch 4/20
396/396 [==============================] - 22s 56ms/step - loss: 0.1161 - accuracy: 0.9668 - val_loss: 0.2305 - val_accuracy: 0.9358
Epoch 5/20
396/396 [==============================] - 22s 57ms/step - loss: 0.0846 - accuracy: 0.9733 - val_loss: 0.2294 - val_accuracy: 0.9387
Epoch 6/20
396/396 [==============================] - 22s 57ms/step - loss: 0.0757 - accuracy: 0.9764 - val_loss: 0.2444 - val_accuracy: 0.9384
Epoch 7/20
396/396 [==============================] - 23s 57ms/step - loss: 0.0550 - accuracy: 0.9845 - val_loss: 0.2311 - val_accuracy: